## Documentation:

### Tech stack:
    1. I use Open CV for working with images and video
    2. I use Image AI as YOLO v3 framework for object detection
        GitHub: https://github.com/OlafenwaMoses/ImageAI/
    3. I use openpyxl for writing Excel files
    4. I use PyQT as User Interface
    5. Deploy via py2exe
    
### Deployment environment:
As packet manager I use Anaconda. Firstly create an environment:
    <pre><code>
    conda create -n DAI-detector -c anaconda keras==2.4.3 numpy==1.19.3 pillow==7.0.0 scipy==1.4.1 h5py==2.10.0 matplotlib==3.3.2 opencv-python keras-resnet==0.2.0 jupyter
    </code></pre>
Next import Tensorflow:
    <pre><code>
    pip install tensorflow==2.4.0
    </code></pre>
    <pre><code>
    pip install tensorflow-gpu==2.4.0
    </code></pre>
Check for updates:
    <pre><code>
    pip install imageai --upgrade
    </code></pre>
Now install libraries:
    <pre><code>
    pip install pandas
    </code></pre>
    <pre><code>
    pip install openpyxl
    </code></pre>
    <pre><code>
    pip install xlswriter
    </code></pre>
    <pre><code>
    pip install pyqt5
    </code></pre>
    <pre><code>
    pip install pyqt5-tools
    </code></pre>
    <pre><code>
    pip install py2exe
    </code></pre>
    
    
## Annotating Data:
I use the labelImg annotation tool:
https://github.com/tzutalin/labelImg#hotkeys

### First cascade detect classes below:
- DAI - the workspace of DAI

### Second cascade detect classes below:

- cMCH - the center of the micron clock hand
- eMCH - the edge of the micron clock hand
- mmWS - mm clock face workzone
- mmCH - mm clock hand

## First version of DAI (distance amplifying instrument) Detector

### Initialize the Libraries

In [ ]:
import cv2 as cv
from imageai.Detection import ObjectDetection

import numpy as np
import requests as req
import os as os
import time
import math

## Train the models

This section need if you want to train your own detection model. If you don't, skip it.

### Training part (first cascade)

In [ ]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="res/first cascade")

trainer.setTrainConfig(object_names_array=["DAI"], batch_size=4, num_experiments=10, 
                       train_from_pretrained_model="pretrained-yolov3.h5")

trainer.trainModel()

### Training part (Second cascade)

In [ ]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="res/second cascade")

trainer.setTrainConfig(object_names_array=["cMCH", "MCH", "mmWS", "mmCH"], batch_size=4, num_experiments=20, 
                       train_from_pretrained_model="pretrained-yolov3.h5")

trainer.trainModel()

## OpenCV Part

### Video slice

Pretty useful script for slicing video into images

In [ ]:
def videoSlice(inputFile, outputFolder, step = 2):
    if os.path.exists(inputFile) and os.path.exists(outputFolder):
        current_data = time.strftime("%d%m%y %H%M%S", time.localtime())
        cap = cv.VideoCapture(inputFile)
        i = 0
        while (cap.isOpened()):
            ret, frame = cap.read()
            for a in range(step-1):
                ret, frame = cap.read()  
            if type(frame) != type(np.array([])):
                print("broken frame")
                break
            
            cropPath = "{0}/{1} {2}.png".format(outputFolder, current_data, i)
            cv.imwrite(cropPath, frame)
            display('proccesed: {0}'.format(i))
            i += 1
            if cv.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        display('Done')
    else:
        return 'no such a file'

videoSlice("INPUT/FILE", "OUTPUT/FILE", step = 5) #Step means use every 1 of $step$ image

### Web-Cam Test

If you dont sure that it works

In [ ]:
cap = cv.VideoCapture(0)
cap.set(3, 640)  # width size
cap.set(4, 480)  # height size
while True:
    succes, img = cap.read();
    if succes:
        cv.imshow("Video", img)
        if cv.waitKey(10) & 0xFF == ord('q'):
            break
    else:
        print("Access denied")

## Here is the programme starts

### Math/Geometry functions

In [ ]:
def orientation(x1, x2, x3, y1, y2, y3): # Orientation of point x3,y3 relative to vector starts in x1,y1 and ends in x2,y2
    x3 -= x1
    y3 -= y1
    a = ((x2 - x1)**2 + (y2- y1)**2)**0.5
    b = x2 - x1
    c = y2 - y1
    if b == 0 or c == 0:
        return 0
    alpha = (b*b + c*c - a*a)/(2*b*c)    
    M = np.array([[alpha, -np.sin(np.arccos(alpha))], [np.sin(np.arccos(alpha)), alpha]]) # Transformation matrix Oxy -> vector's basis    
    #X3 = M[0,0] * x3 + M[0,1] * y3
    Y3 = M[1,0] * x3 + M[1,1] * y3    
    if Y3 > 0:
        return 1
    else:
        return 0
    
def cosTh(x3, x2, x1, y3, y2, y1): # Angle between vectors c = (x3 - x1, y3 - y1) and b = (x3 - x2, y3 - y2)
    a = ((x2 - x1)**2 + (y2- y1)**2)**0.5
    b = ((x3 - x1)**2 + (y3- y1)**2)**0.5
    c = ((x3 - x2)**2 + (y3- y2)**2)**0.5
    
    if b == 0 or c == 0:
        return 0
    alpha = (b*b + c*c - a*a)/(2*b*c)
    if orientation(x1, x3, x2, y1, y3, y2):
        alpha = 180 - np.arccos(alpha)*180/np.pi
    else:
        alpha = 180 + np.arccos(alpha)*180/np.pi
    return alpha


def scale(x, y, w, h, scaleKoef = 0.1):
    hh = round(scaleKoef * (h - y))
    ww = round(scaleKoef * (w - x))
    if y - hh > 0 and x - ww > 0:
        y -= hh
        x -= ww
        h += hh
        w += ww
    return x, y, w, h

def drawLine(image, x1, x2, y1, y2, koef = 2.5, line_thickness = 2, approx = 0):
    cv.line(image, (x2, y2), (x1, y1), (0, 255, 0), thickness=line_thickness)
    if approx:
        dX = x2 - x1
        dY = y2 - y1
        dX = round(x1 - dX * koef)
        dY = round(y1 - dY * koef)
        cv.line(image, (dX, dY), (x1, y1), (0, 0, 255), thickness=line_thickness)
    return image
    

### Detection functions

In [ ]:
from imageai.Detection.Custom import CustomObjectDetection

class DAIdetector:
    
    def __init__(self, fstProb = 50, secProb = 50):
        self.fstProb = fstProb
        self.secProb = secProb
        
        self.detector = CustomObjectDetection()
        self.detector.setModelTypeAsYOLOv3()
        self.detector.setModelPath("res/first cascade/models/detection_model-ex-015--loss-0013.902.h5") #Model file
        self.detector.setJsonPath("res/first cascade/json/detection_config.json") #Its JSON file
        self.detector.loadModel()

        self.AngleDetector = CustomObjectDetection()
        self.AngleDetector.setModelTypeAsYOLOv3()
        self.AngleDetector.setModelPath("res/second cascade/models/detection_model-ex-015--loss-0022.383.h5") #Model file
        self.AngleDetector.setJsonPath("res/second cascade/json/detection_config.json") #Its JSON file
        self.AngleDetector.loadModel()
        pass
    
    def detectDAI(self, inputImg, inputType = "file"):       
        inputImg = inputImg.copy()
        detectedImage, detections = self.detector.detectObjectsFromImage(output_type="array", input_type = inputType,
                                                                         input_image = inputImg, 
                                                                         minimum_percentage_probability = self.fstProb)
        DAI, DAIcoord, alpha, DAIimgSet = np.array([]), np.array([]), np.array([]), []
    
        i = 0
           
        for eachDAI in detections:
            if eachDAI["name"] == "DAI":
                eachDAI["name"] = "DAI {0}".format(i)
                x,y,w,h = eachDAI["box_points"]
                #x,y,w,h = scale(x,y,w,h)
                cropImg = inputImg[y : h, x : w] 
                cropImg = cv.resize(cropImg, (320, 320))
                
                current_time = time.strftime("%d%m%y %H%M%S", time.localtime())
                cropPath = "res/buffer/SecCasImg/{0} {1}.jpg".format(current_time, i)
                cv.imwrite(cropPath, cropImg)
            
                Angles, detAngles = self.AngleDetector.detectObjectsFromImage(output_type='array', input_type = 'array',
                                                                              input_image = cropImg,
                                                                              minimum_percentage_probability = self.secProb)
                procIMG, a = self.secondCascade(cropImg, detAngles)
                if a == None:
                    a = 'Can not detect'
                    procIMG = Angles
                DAI = np.append(DAI, eachDAI["name"])
                DAIcoord = np.append(DAIcoord, eachDAI["box_points"])
                alpha = np.append(alpha, a)
                DAIimgSet.append(procIMG) 
                i += 1

        return detectedImage, DAI, DAIcoord, alpha, DAIimgSet

    def secondCascade(self, Angles, detAngles):
        x1, x2, x3, x4, y1, y2, y3, y4 = -1,-1,-1,-1,-1,-1,-1,-1
        cMCH, MCH, mmWS, mmCH = 0,0,0,0
        for el in detAngles:
            if el['name'] == 'cMCH' and el['percentage_probability'] > cMCH:
                cMCH = el['percentage_probability']
                x1, y1, a, b = el['box_points']
                x1, y1 = round((x1 + a)/2), round((y1 + b)/2)
            elif el['name'] == 'MCH'and el['percentage_probability'] > MCH:
                MCH = el['percentage_probability']
                x2, y2, a, b = el['box_points']
                x2, y2 = round((x2 + a)/2), round((y2 + b)/2)
            elif el['name'] == 'mmWS'and el['percentage_probability'] > mmWS:
                mmWS = el['percentage_probability']
                x3, y3, a, b = el['box_points']
                x3, y3 = round((x3 + a)/2), round((y3 + b)/2)
            elif el['name'] == 'mmCH'and el['percentage_probability'] > mmCH:
                mmCH = el['percentage_probability']
                x4, y4, a, b = el['box_points']
                x4, y4 = round((x4 + a)/2), round((y4 + b)/2)
        if x1 == -1 or x2 == -1 or x3 == -1:
            return Angles, None
            
        drawLine(Angles, x1, x2, y1, y2, approx = 1)
        micronAngle = cosTh(x1, x2, x3, y1, y2, y3)
        
        org = (6, 15)
        fontScale = 0.4
        thickness = 1
        
        micronAngle = round(micronAngle / 180 * 50) #angle to microns
        if micronAngle == 100:
            micronAngle = 0
        if x4 != -1:
            drawLine(Angles, x3, x4, y3, y4)
            mmAngle = cosTh(x3, x4, x1, y3, y4, y1)
            if micronAngle < 50 and (mmAngle - math.floor(mmAngle) >= 20):
                mmAngle = math.ceil(mmAngle / 180 * 5) #angle to mm
            else:
                mmAngle = round(mmAngle / 180 * 5) #angle to mm
            if mmAngle >= 10 or mmAngle <= 0:
                mmAngle = 0
            image = cv.putText(Angles, 'a= {0} mm'.format(round(mmAngle + micronAngle/100, 2)), org, cv.FONT_HERSHEY_SIMPLEX, 
                           fontScale, (0,0,255), thickness, cv.LINE_AA)
            return image, mmAngle + micronAngle/100
        else:
            if micronAngle >= 10:
                image = cv.putText(Angles, 'a=?.{0} mm'.format(micronAngle), org, cv.FONT_HERSHEY_SIMPLEX, 
                           fontScale, (0,0,255), thickness, cv.LINE_AA)
                return image, '?.{0}'.format(micronAngle)
            else:
                image = cv.putText(Angles, 'a=?.0{0} mm'.format(micronAngle), org, cv.FONT_HERSHEY_SIMPLEX, 
                           fontScale, (0,0,255), thickness, cv.LINE_AA)
                return image, '?.0{0}'.format(micronAngle)
        

### Output functions

In [ ]:
import pandas as pd
import openpyxl 

class ExcelPrinter:
    
    def __init__(self, directory = "res/output", fileName = "DAI"):
        self.directory = directory
        self.fileName = fileName
        self.data = pd.DataFrame({'Data': [], 'Time': [], 'DAI' : [], 'DAI coordinates: x' : [],
                                'DAI coordinates: y' : [], 'DAI coordinates: w' : [], 'DAI coordinates: h' : [],
                                'Angle' : []})
        pass
    
    def addObservation(self, DAI, DAIcoord, angles): 
        t = time.localtime()
        current_time = time.strftime("%X", t)
        current_data = time.strftime("%x", t)
        for i in range(len(DAI)):
            name = DAI[i]
            a = angles[i]
            x = DAIcoord[4 * i]
            y = DAIcoord[4 * i + 1]
            w = DAIcoord[4 * i + 2]
            h = DAIcoord[4 * i + 3]
            new_row = {'Data': current_data,'Time': current_time, 'DAI' : name, 'DAI coordinates: x' : x,
                        'DAI coordinates: y' : y, 'DAI coordinates: w' : w, 'DAI coordinates: h' : h, 'Angle' : a}
            self.data = self.data.append(new_row, ignore_index=True)
        with pd.ExcelWriter("{0}/{1}.xlsx".format(self.directory, self.fileName), mode="w", engine='openpyxl') as writer:
            self.data.to_excel(writer, sheet_name="DAI")
            
        
    def newfile(self):
        with pd.ExcelWriter("{0}/DAI.xlsx".format(self.directory), engine='xlsxwriter') as writer:
            self.data.to_excel(writer, sheet_name='DAI')
            writer.save()


## User Interface:

In [ ]:
import sys
import os
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QFileDialog, QApplication, QWidget, QLabel, QMessageBox
from PyQt5.QtGui import QImage, QIcon, QPixmap

import cv2 as cv
import time 

class Ui_MainWindow(object):
    
    def __init__(self):
        self.fstProbability = 50 # First cascade probability value
        self.secProbability = 50 # Second cascade probability value
        self.detectionModel = DAIdetector(fstProb = self.fstProbability, secProb = self.secProbability)
        self.outputResolution = (500,500)
        self.workStatus = False
        self.saveFolder = 'res/output'
        self.imgStream = None
        self.fileName = None # Will hold the image address location
        self.origTmp = None # Original image
        self.procTmp = None # Proccesed image
        self.camPort = 0 # Camera port
        pass
    
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(800, 600)
        MainWindow.setMinimumSize(QtCore.QSize(800, 600))
        MainWindow.setMaximumSize(QtCore.QSize(800, 600))
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        self.verticalLayoutWidget = QtWidgets.QWidget(self.centralwidget)
        self.verticalLayoutWidget.setGeometry(QtCore.QRect(0, 0, 631, 541))
        self.verticalLayoutWidget.setObjectName("verticalLayoutWidget")
        self.showStack = QtWidgets.QVBoxLayout(self.verticalLayoutWidget)
        self.showStack.setContentsMargins(0, 0, 0, 0)
        self.showStack.setObjectName("showStack")
        self.ImageOutputTabs = QtWidgets.QTabWidget(self.verticalLayoutWidget)
        self.ImageOutputTabs.setObjectName("ImageOutputTabs")
        self.origTab = QtWidgets.QWidget()
        self.origTab.setMinimumSize(QtCore.QSize(500, 500))
        self.origTab.setObjectName("origTab")
        self.origImg = QtWidgets.QLabel(self.origTab)
        self.origImg.setGeometry(QtCore.QRect(0, 0, 500, 500))
        self.origImg.setFrameShape(QtWidgets.QFrame.Box)
        self.origImg.setFrameShadow(QtWidgets.QFrame.Plain)
        self.origImg.setText("")
        self.origImg.setObjectName("origImg")
        self.ImageOutputTabs.addTab(self.origTab, "")
        self.procTab = QtWidgets.QWidget()
        self.procTab.setObjectName("procTab")
        self.procImg = QtWidgets.QLabel(self.procTab)
        self.procImg.setGeometry(QtCore.QRect(0, 0, 500, 500))
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Maximum, QtWidgets.QSizePolicy.Maximum)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.procImg.sizePolicy().hasHeightForWidth())
        self.procImg.setSizePolicy(sizePolicy)
        self.procImg.setFrameShape(QtWidgets.QFrame.Box)
        self.procImg.setFrameShadow(QtWidgets.QFrame.Plain)
        self.procImg.setText("")
        self.procImg.setTextFormat(QtCore.Qt.AutoText)
        self.procImg.setAlignment(QtCore.Qt.AlignCenter)
        self.procImg.setWordWrap(False)
        self.procImg.setObjectName("procImg")
        self.ImageOutputTabs.addTab(self.procTab, "")
        self.showStack.addWidget(self.ImageOutputTabs)
        self.InputStreamTabs = QtWidgets.QTabWidget(self.centralwidget)
        self.InputStreamTabs.setGeometry(QtCore.QRect(640, 179, 160, 301))
        self.InputStreamTabs.setObjectName("InputStreamTabs")
        self.file = QtWidgets.QWidget()
        self.file.setObjectName("file")
        self.fileImputButton = QtWidgets.QPushButton(self.file)
        self.fileImputButton.setGeometry(QtCore.QRect(0, 10, 150, 28))
        sizePolicy = QtWidgets.QSizePolicy(QtWidgets.QSizePolicy.Minimum, QtWidgets.QSizePolicy.Minimum)
        sizePolicy.setHorizontalStretch(0)
        sizePolicy.setVerticalStretch(0)
        sizePolicy.setHeightForWidth(self.fileImputButton.sizePolicy().hasHeightForWidth())
        self.fileImputButton.setSizePolicy(sizePolicy)
        self.fileImputButton.setObjectName("fileImputButton")
        self.InputStreamTabs.addTab(self.file, "")
        self.cam = QtWidgets.QWidget()
        self.cam.setObjectName("cam")
        self.cameraPort = QtWidgets.QComboBox(self.cam)
        self.cameraPort.setGeometry(QtCore.QRect(10, 10, 130, 22))
        self.cameraPort.setObjectName("cameraPort")
        self.cameraPort.addItem("")
        self.cameraPort.addItem("")
        self.cameraPort.addItem("")
        self.cameraPort.addItem("")
        self.cameraPort.addItem("")
        self.cameraPort.addItem("")
        self.cameraPort.addItem("")
        self.cameraPort.addItem("")
        self.cameraPort.addItem("")
        self.cameraPort.addItem("")
        self.InputStreamTabs.addTab(self.cam, "")
        self.Text = QtWidgets.QLabel(self.centralwidget)
        self.Text.setGeometry(QtCore.QRect(640, 0, 160, 40))
        font = QtGui.QFont()
        font.setFamily("Times New Roman")
        font.setPointSize(12)
        self.Text.setFont(font)
        self.Text.setAlignment(QtCore.Qt.AlignCenter)
        self.Text.setObjectName("Text")
        self.probabilitySliderFirst = QtWidgets.QSlider(self.centralwidget)
        self.probabilitySliderFirst.setGeometry(QtCore.QRect(640, 60, 151, 22))
        self.probabilitySliderFirst.setMinimum(1)
        self.probabilitySliderFirst.setProperty("value", 50)
        self.probabilitySliderFirst.setOrientation(QtCore.Qt.Horizontal)
        self.probabilitySliderFirst.setObjectName("probabilitySliderFirst")
        self.startButton = QtWidgets.QPushButton(self.centralwidget)
        self.startButton.setGeometry(QtCore.QRect(640, 480, 160, 30))
        self.startButton.setObjectName("startButton")
        self.stopButton = QtWidgets.QPushButton(self.centralwidget)
        self.stopButton.setGeometry(QtCore.QRect(640, 510, 160, 30))
        self.stopButton.setObjectName("stopButton")
        self.fileOutputButton = QtWidgets.QPushButton(self.centralwidget)
        self.fileOutputButton.setGeometry(QtCore.QRect(640, 140, 150, 28))
        self.fileOutputButton.setObjectName("fileOutputButton")
        self.probabilitySliderSecond = QtWidgets.QSlider(self.centralwidget)
        self.probabilitySliderSecond.setGeometry(QtCore.QRect(640, 110, 151, 22))
        self.probabilitySliderSecond.setMinimum(1)
        self.probabilitySliderSecond.setProperty("value", 50)
        self.probabilitySliderSecond.setOrientation(QtCore.Qt.Horizontal)
        self.probabilitySliderSecond.setObjectName("probabilitySliderSecond")
        self.label = QtWidgets.QLabel(self.centralwidget)
        self.label.setGeometry(QtCore.QRect(640, 40, 150, 16))
        self.label.setObjectName("label")
        self.label_2 = QtWidgets.QLabel(self.centralwidget)
        self.label_2.setGeometry(QtCore.QRect(640, 90, 141, 16))
        self.label_2.setObjectName("label_2")
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 800, 26))
        self.menubar.setObjectName("menubar")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        self.ImageOutputTabs.setCurrentIndex(0)
        self.InputStreamTabs.setCurrentIndex(1)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)
        
        self.add_functions()
        self.reset()

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "DAI detector"))
        self.ImageOutputTabs.setTabText(self.ImageOutputTabs.indexOf(self.origTab), _translate("MainWindow", "Original Image"))
        self.ImageOutputTabs.setTabText(self.ImageOutputTabs.indexOf(self.procTab), _translate("MainWindow", "Proccesed Image"))
        self.fileImputButton.setText(_translate("MainWindow", "Input File"))
        self.InputStreamTabs.setTabText(self.InputStreamTabs.indexOf(self.file), _translate("MainWindow", "File"))
        self.cameraPort.setItemText(0, _translate("MainWindow", "1"))
        self.cameraPort.setItemText(1, _translate("MainWindow", "2"))
        self.cameraPort.setItemText(2, _translate("MainWindow", "3"))
        self.cameraPort.setItemText(3, _translate("MainWindow", "4"))
        self.cameraPort.setItemText(4, _translate("MainWindow", "5"))
        self.cameraPort.setItemText(5, _translate("MainWindow", "6"))
        self.cameraPort.setItemText(6, _translate("MainWindow", "7"))
        self.cameraPort.setItemText(7, _translate("MainWindow", "8"))
        self.cameraPort.setItemText(8, _translate("MainWindow", "9"))
        self.cameraPort.setItemText(9, _translate("MainWindow", "10"))
        self.InputStreamTabs.setTabText(self.InputStreamTabs.indexOf(self.cam), _translate("MainWindow", "Camera"))
        self.Text.setText(_translate("MainWindow", "Min % probability"))
        self.startButton.setText(_translate("MainWindow", "START"))
        self.stopButton.setText(_translate("MainWindow", "STOP"))
        self.fileOutputButton.setText(_translate("MainWindow", "Output File"))
        self.label.setText(_translate("MainWindow", "First cascade: 50"))
        self.label_2.setText(_translate("MainWindow", "Second cascade: 50"))


    def reset(self):
        self.fstProbability = 50 # First cascade probability value
        self.secProbability = 50 # Second cascade probability value
        self.detectionModel = DAIdetector(fstProb = self.fstProbability, secProb = self.secProbability)
        self.outputResolution = (500,500)
        self.workStatus = False
        self.saveFolder = 'res/output'
        self.imgStream = None
        self.fileName = None # Will hold the image address location
        self.origTmp = None # Original image
        self.procTmp = None # Proccesed image
        self.camPort = 0 # Camera port
    
    def add_functions(self):
        self.startButton.clicked.connect(self.start)
        self.stopButton.clicked.connect(self.stop)
        self.fileImputButton.clicked.connect(self.loadVideo)
        self.fileOutputButton.clicked.connect(self.setSaveFolder)
        self.probabilitySliderFirst.valueChanged['int'].connect(self.firstProbValue)
        self.probabilitySliderSecond.valueChanged['int'].connect(self.secondProbValue)
        
    def erMessage(self, winName, erText):
        msg = QMessageBox()
        msg.setWindowTitle(winName)
        msg.setText(erText)
        msg.setIcon(QMessageBox.Warning)
        msg.setStandardButtons(QMessageBox.Ok)
        msg.exec_()
        
    def start(self):   
        output = ExcelPrinter(directory = self.saveFolder)
        output.newfile()
        frame_num = 0
        start_time = time.time()
        fps = 0
        self.workStatus = True
        try:
            fourcc = cv.VideoWriter_fourcc(*'XVID')
            vidout = cv.VideoWriter('{0}/{1}.mp4'.format(self.saveFolder, 'Proccesed video'),
                                    fourcc, 20.0, (500, 500))

            er = 0
            while (self.imgStream.isOpened()):  
                ret, frame = self.imgStream.read()  
                if type(frame) != type(np.array([])):
                    if er > 3:
                        break
                    er += 1
                    print("broken frame")
                    continue

                image, DAI, DAIcoord, alpha, DAIimgSet = self.detectionModel.detectDAI(frame, inputType = "array")

                end_time = time.time()
                fps = fps * 0.9 + 1/(end_time - start_time) * 0.1
                start_time = end_time
                # Draw additional info
                image = cv.resize(image, (500,500))
                frame_info = 'Frame: {0}, FPS: {1:.2f}'.format(frame_num, fps)
                cv.putText(image, frame_info, (10, image.shape[0]-10),
                            cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

                outImage = self.multiStack(DAIimgSet)
                self.setOrigFrame(image)
                self.setProcFrame(outImage)

                print(DAI, DAIcoord, alpha)
                if len(DAIcoord) != 0:
                    output.addObservation(DAI, DAIcoord, alpha)
                vidout.write(cv.flip(cv.flip(outImage,0), 0))

                key = cv.waitKey(1) & 0xFF

                # Exit
                if not self.workStatus:
                    break

                # Take screenshot
                if key == ord('s'):
                    cv.imwrite('{0}/frame_{1}.jpg'.format(self.saveFolder, time.time()), frame)

                frame_num += 1
            display('Done')
        except BaseException:
            self.erMessage('Error', str(BaseException))
        else:
            self.erMessage('working status', 'Computations complete')
        finally:
            self.workStatus = False
            self.imgStream.release()
            vidout.release()
    
    def stop(self):
        self.workStatus = False
        pass
        
    def loadVideo(self):
        file = QFileDialog.getOpenFileName(filter="Video (*.*)")[0]
        if not os.path.exists(file):
            raise IOError('Can\'t open "{0}"'.format(file))
        self.imgStream = cv.VideoCapture(file)
        fourcc = cv.VideoWriter_fourcc(*'XVID')
        #self.video = cv.imread(self.fileName)
        print('File choosen: {0}'.format(file))
        self.update()

    def setOrigFrame(self,image):
        self.origTmp = image
        image = cv.resize(image, (500, 500))
        frame = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        image = QImage(frame, frame.shape[1],frame.shape[0],frame.strides[0],QImage.Format_RGB888)
        self.origImg.setPixmap(QtGui.QPixmap.fromImage(image))

    def setProcFrame(self,image):
        self.procTmp = image
        image = cv.resize(image, (500, 500))
        frame = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        image = QImage(frame, frame.shape[1],frame.shape[0],frame.strides[0],QImage.Format_RGB888)
        self.procImg.setPixmap(QtGui.QPixmap.fromImage(image))
        
    def setSaveFolder(self):
        self.saveFolder = QFileDialog.getExistingDirectory()
        print('Image saved in: {0}'.format(self.saveFolder))
        
    def firstProbValue(self, value):
        self.fstProbability = value
        print('First cascade probability: ',value)
        self.update()
        
    def secondProbValue(self, value):
        self.secProbability = value
        print('Second cascade probability: ',value)
        self.update()
    
    def update(self):
        self.label.setText("First cascade: {0}".format(self.fstProbability))
        self.label_2.setText("Second cascade: {0}".format(self.secProbability))
        if self.workStatus == True:
            self.detectionModel.fstProb = self.fstProbability 
            self.detectionModel.secProb = self.secProbability
        #self.setProcFrame(img)
        #self.setOrigFrame(img2)
        pass
    
    def multiStack(self, imgArray):
        rows =  math.ceil(len(imgArray) ** 0.5)
        delta = rows * rows - len(imgArray)
        print("{0} of {1} used".format(len(imgArray), rows))
        imgStack = []

        if isinstance(imgArray, list):
            if len(imgArray) == 0:
                gray_level = 127
                gray_image = gray_level * np.ones((self.outputResolution[0], self.outputResolution[1], 3), dtype = np.uint8)
                return gray_image
            for x in range(0, rows):
                result = []
                for y in range(0, rows):
                    if x * rows + y < len(imgArray):
                        resized = cv.resize(imgArray[x * rows + y], 
                                            (round(self.outputResolution[0]/rows),round(self.outputResolution[1]/rows)))
                        result.append(resized)
                    else:
                        gray_level = 127
                        gray_image = gray_level * np.ones((round(
                            self.outputResolution[0]/rows),round(self.outputResolution[1]/rows),3), dtype = np.uint8)
                        result.append(gray_image)
                imgStack.append(np.hstack(result))
            return np.vstack(imgStack)
        else:
            assert 'Wrong Image Array type in stacking function'


if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())